In [45]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [46]:
data = pd.read_csv('train.csv')
#, encoding="utf-8"

In [47]:
# Normalize the pixel values
data = np.array(data)
data[:, 1:] = data[:, 1:] / 255.0

In [48]:
# Shuffle the data
np.random.shuffle(data)

C:\Users\gayan\AppData\Local\Temp\ipykernel_20048\1757574140.py:2: UserWarning: you are shuffling a 'DataFrame' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(data)


KeyError: 32070

In [ ]:
# Split the data into training and development sets
data_dev = data[:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:]

data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:]

In [ ]:
# Initializing parameters

def init_params():
    W1 = np.random.rand(10, 784)
    b1 = np.random.rand(10, 1)
    W2 = np.random.rand(10, 10)
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2
    

In [ ]:
# forward propagation

def ReLU(Z):
    return np.maximum(0, Z)

# def softmax(Z):
#     return np.exp(Z)/np.sum(np.exp(Z))

def stable_softmax(Z):
    Z -= np.max(Z)  # Subtract the maximum value for stability
    exp_Z = np.exp(Z)
    return exp_Z / np.sum(exp_Z)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = stable_softmax(Z2)
    return Z1, A1, Z2, A2

In [ ]:
# backward propagation

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [ ]:
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 10 == 0):
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f'Iteration: {i}, Accuracy: {accuracy:.4f}')

    return W1, b1, W2, b2

# Increase the number of iterations and adjust the learning rate as needed
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 300, 0.01)